# 配对

## 最小距离 

In [1]:
# 收益比相差最小

from lib.utils.lmdb_dataset import get_dataset
from root import get_root_path
import lib.utils.sdata as sd
import importlib
import os
import numpy as np
import lib.model.pairs_model as pm
importlib.reload(sd)
importlib.reload(pm)


main_code = '600893'
other_code = ['000998','002261','000955','600309','600893']
db_path = os.path.sep.join([get_root_path(),'lmdb'])
pmodel = pm.PairTrading()

main_datas = get_dataset(db_path,main_code)
# print(main_datas.head(5))
for icode in other_code:
    other_datas = get_dataset(db_path, icode)
    instance = pmodel.SSD(main_datas.Close, other_datas.Close)
    print('code {}'.format(icode), instance)
    

code 000998 26.65603913570967
code 002261 44.36596668620415
code 000955 29.32107150339057
code 600309 27.566102713911494
code 600893 0.0


## 相关性，协方差比较 

### 数据准备 

In [19]:
#  https://blog.csdn.net/weixin_34111790/article/details/89622651  相关性corr，协方差 cov
#  https://blog.csdn.net/qq_40946639/article/details/102984166     pandas——相关系数函数corr()
'''
协方差: 如果返回的是正数，代表两组数据“同向”发展,数值越大，“同向”程度越高 , 如为负数，则“反向”程序越高
相关系数(判断两组数据是否相似): 取值为[-1,1] 越接近1，代表相似程度越高
'''
from lib.utils.lmdb_dataset import get_dataset
from root import get_root_path
import lib.utils.sdata as sd
import importlib
import os
import numpy as np
import lib.model.pairs_model as pm
import pandas as pd
importlib.reload(sd)
importlib.reload(pm)

codes = ['000998','002261','000955','600309','600893']
db_path = os.path.sep.join([get_root_path(),'lmdb'])
datagroups = None

for icode in codes:
    print(icode)
    data = get_dataset(db_path, icode)
    price = data.Close
    #  收益
    priceReturn = (price - price.shift(1)) / price.shift(1)[1:]
    priceReturnDf = pd.DataFrame(index=data.index, data=priceReturn.values, columns=[icode])
    if datagroups is None:
        datagroups = priceReturnDf.copy();
    else:
        datagroups = pd.merge(datagroups,priceReturnDf,left_index=True, right_index=True)

000998
002261
000955
600309
600893


### 相关性分析

In [26]:
data_corr = datagroups.corr()
print("相关性-->\n" , data_corr)

data_cov = datagroups.cov()
print('\n协方差-->\n', data_cov)

相关性-->
           000998    002261    000955    600309    600893
000998  1.000000  0.312504  0.324196  0.353775  0.328119
002261  0.312504  1.000000  0.371431  0.277506  0.342566
000955  0.324196  0.371431  1.000000  0.375740  0.356382
600309  0.353775  0.277506  0.375740  1.000000  0.333643
600893  0.328119  0.342566  0.356382  0.333643  1.000000

协方差-->
           000998    002261    000955    600309    600893
000998  0.000956  0.000376  0.000305  0.000293  0.000333
002261  0.000376  0.001513  0.000439  0.000289  0.000437
000955  0.000305  0.000439  0.000925  0.000306  0.000355
600309  0.000293  0.000289  0.000306  0.000719  0.000293
600893  0.000333  0.000437  0.000355  0.000293  0.001075


### 网络结点图